# Steel Plate Defect Prediction

In [48]:
%%time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputClassifier

import os
import warnings
warnings.filterwarnings('ignore')

CPU times: user 51 µs, sys: 1e+03 ns, total: 52 µs
Wall time: 55.1 µs


In [49]:
#original_data = pd.read_csv('data/train.csv',index_col=0)
train = pd.read_csv('data/train.csv',index_col=0)
test = pd.read_csv('data/test.csv',index_col=0)
submission = pd.read_csv('data/sample_submission.csv',index_col=0)

In [50]:
train.shape, test.shape, submission.shape

((19219, 34), (12814, 27), (12814, 7))

In [51]:
target_columns = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains','Dirtiness', 'Bumps', 'Other_Faults']

X = train.drop(target_columns, axis=1)
y = train[target_columns]